In [22]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

C:\Users\HP\anaconda3\envs\rossmann\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [23]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import load_model

In [24]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

In [9]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']


In [25]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday']
# contin_vars = []

In [33]:
cat_vars = ['Store', 'DayOfWeek' ]+['Year', 'Month', 'Day', 'StateHoliday', 
            'StoreType', 'Assortment','Week', 'Events', 'Promo_fw', 'CompetitionOpenSinceYear']
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Precipitationmm',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE','AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday']

In [6]:
y_out_columns = ['Sales']

In [26]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [27]:
X_train = df_train[cat_vars + contin_vars]
X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [28]:
X_train.shape, X_val.shape

((814150, 38), (30188, 38))

In [29]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    y_val = (df_val[y_out_columns].values - y_mean)/y_std

## Categorical columns transformation

In [30]:
# categoricals_processing = 'no_categoricals'
# categoricals_processing = 'use_onehotencoding'
categoricals_processing = 'use_embeddings'


In [32]:
if categoricals_processing == 'use_embeddings':
    embeddings_model = load_model('embeddings_modelfound.hdf5')
    X_train = embeddings_model.predict(np.hsplit(X_train, X_train.shape[1]), verbose=1)
    X_val = embeddings_model.predict(np.hsplit(X_val, X_val.shape[1]), verbose=1)
    X_test = embeddings_model.predict(np.hsplit(X_test, X_test.shape[1]), verbose=1)

41088/41088 [==============================] - 6s 136us/sample


In [34]:
X_train.shape

(814150, 134)

In [13]:
if categoricals_processing == 'use_onehotencoding':
    # Use One Hot Encoding
    categorical_feature_mask = [col in cat_vars for col in X_train.columns]
    ohe = OneHotEncoder(categorical_features = categorical_feature_mask)
    ohe.fit(X_train)
    X_train = ohe.transform(X_train)
    X_val = ohe.transform(X_val)
    X_test = ohe.transform(X_test)

In [35]:
X_train.shape

(814150, 134)

In [36]:
X_val.shape

(30188, 134)

In [37]:
X_test.shape

(41088, 134)

## XGBoost

In [38]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

### Fine Tune

In [43]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [48]:
params = { 'max_depth': [6, 8, 10],
           'learning_rate': [ 0.05],
           #'subsample': np.arange(0.5, 1.0, 0.1),
           #'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           #'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           #'min_child_weight':
           'n_estimators': [3000, 4000, 5000]}

In [49]:
xgbr = xgb.XGBRegressor(seed = 20)

In [67]:
clf = RandomizedSearchCV(estimator=xgbr,
                         cv=3,
                         param_distributions=params,
                         scoring='neg_mean_squared_error',
                         n_iter=10,
                         verbose=1)


In [107]:
clf.fit(X_train, y_train.reshape(-1))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\HP\anaconda3\envs\rossmann\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_10328\1998294088.py", line 1, in <module>
    clf.fit(X_train, y_train.reshape(-1))
  File "C:\Users\HP\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py", line 891, in fit
    self._run_search(evaluate_candidates)
  File "C:\Users\HP\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py", line 1768, in _run_search
    self.param_distributions, self.n_iter, random_state=self.random_state
  File "C:\Users\HP\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py", line 851, in evaluate_candidates
    enumerate(candidate_params), enumerate(cv.split(X, y, groups))
  File "C:\Users\HP\AppData\Roaming\Python\Python37\site-packages\joblib\parallel.py", lin

TypeError: object of type 'NoneType' has no len()

In [ ]:
#modelos_xgboost = pd.DataFrame(clf.cv_results_)
#modelos_xgboost = modelos_lg.sort_values('rank_test_score',ascending=True).head()
#modelos_xgboost

In [106]:
print("Best parameters:", clf.best_estimator_)
#print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

### Estimation

In [108]:
n_estimators=4000
learning_rate=0.025
model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate, 
                     objective='reg:squarederror', n_jobs=-1, max_depth=6, 
                     gamma=0.5,subsample=0.4, reg_lambda=1)
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=0.5, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.025, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=4000, n_jobs=-1,
             num_parallel_tree=None, predictor=None, random_state=None,
             reg_alpha=None, reg_lambda=1, ...)

In [94]:
fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'rmse', 
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'verbose': 100,
           }

In [109]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

C:\Users\HP\anaconda3\envs\rossmann\lib\site-packages\xgboost\sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,
C:\Users\HP\anaconda3\envs\rossmann\lib\site-packages\xgboost\sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[0]	validation_0-rmse:0.31958
[100]	validation_0-rmse:0.03248
[200]	validation_0-rmse:0.01571
[300]	validation_0-rmse:0.01511
[400]	validation_0-rmse:0.01504
[500]	validation_0-rmse:0.01504
[600]	validation_0-rmse:0.01503
[700]	validation_0-rmse:0.01503
[800]	validation_0-rmse:0.01503
[812]	validation_0-rmse:0.01503


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.5, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.025, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=4000,
             n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [57]:
# feature importance

#print(model.feature_importances_)
# plot
#pyplot.bar(range(len(model.feature_importances_)), model.feature_importances_)
#pyplot.show()

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [110]:
score = model.score(X_val, y_val)
score

0.8417822620160971

In [89]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train)*max_log_y)
    y_pred = np.exp(model.predict(X_val)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test)*max_log_y)
else:
    y_pred_train = model.predict(X_train)*y_std + y_mean
    y_pred = model.predict(X_val)*y_std + y_mean
    y_pred_test = model.predict(X_test)*y_std + y_mean

In [90]:
# Train
train_RMSE = np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

In [91]:
# Validación
val_RMSE = np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

In [92]:
print(score, train_RMSE, val_RMSE)

0.9098945677867019 0.1704227350987114 0.12548494537086038


In [64]:
model.save_model("model_xgb.json")

In [65]:
model_xgb_2 = xgb.Booster()
model_xgb_2.load_model("model_xgb.json")

# Baseline

In [75]:
import pandas as pd
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')

In [76]:
stores_mean = {}
for store, g_df in df.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [77]:
df_test['Sales'] = df_test['Store'].apply(stores_mean.get)
df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [78]:
df_test[['Store', 'Sales']].head(10)

,Store,Sales
0,0,4759.096031
1,2,6942.568678
2,6,8817.050891
3,7,5539.358418
4,8,6562.337612
5,9,5568.420918
6,10,8030.977041
7,11,7589.598214
8,12,5034.747182
9,13,5508.567394


In [79]:
df_test[df_test['Open'] == 0][['Store', 'Sales']].head()

,Store,Sales
543,702,0.0
676,878,0.0
840,1096,0.0
1399,702,0.0
1532,878,0.0


In [80]:
sample_csv['Sales'] = df_test['Sales']

In [81]:
sample_csv.to_csv(f'submision_baseline.csv', index=False)

In [82]:
sample_csv.head()

,Id,Sales
0,1,4759.096031
1,2,6942.568678
2,3,8817.050891
3,4,5539.358418
4,5,6562.337612


# Sumbit a la competición

In [84]:
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_{log_output}-{n_estimators}-{learning_rate}.csv', index=False)

In [ ]:
cat_vars